In [22]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook
from time import sleep
try:
    import cPickle as pickle     #it is faster than pickle!
except:
    import pickle


In [23]:
test_string="test_"  # or ""

path="../Data/WoS_data/Disambiguated_authors/"

list_years=[2014,2015]
list_df=[]
for year in list_years:
    
    df_aux= pd.read_table('../Data/WoS_data/Disambiguated_authors/'+test_string+str(year)+'.txt',header=None,names=['uid','author_id','author_name','affiliation','idx'])
    list_df.append(df_aux)
    
# concatenate two dataframes with exactly the same column structure
df = pd.concat(list_df)



In [24]:
df.head(5)
#len(df)print len(df_a), len(df_b),  len(df)


,uid,author_id,author_name,affiliation,idx
0,WOS:000333129600024,20140647|33900476|19778882|8710547|1328775|344...,"Yoshikawa, Katsuhisa|Tamai, Kazuya|Yano, Yuich...","Dokkyo Med Univ, Sch Med, Dept Orthoped Surg, ...",0|0|0|0|0|0|0
1,WOS:000332939000004,9664635,"Bordon, Yvonne",NaN,-1
2,WOS:000333069800001,45673005|5470826|3252077,"Lemer, Claire|Cheung, Ronny|Davies, Sally","Evelina Childrens Hosp, Dept Gen Paediat, Lond...",0|1|2
3,WOS:000333001400025,1840092|23730929|16647037|27728784|22798714|33...,"Cao, Jianyu|Wang, Liangduo|Song, Lingzheng|Xu,...","Changzhou Univ, Sch Petrochem Engn, Changzhou ...","0|0|0,1|0|0|0|1|1"
4,WOS:000333000200009,50350675|57842385,"Masalo, I.|Oca, J.","Univ Politecn Catalunya BARCELONATECH, Dept En...",0|0


In [47]:
# unpack each original row into as many rows as authors in the paper:

def process(row):
    
    aids = [int(a) for a in row.author_id.split('|')] # list of author ids in that row
    n = len(aids)
    names = row.author_name.split('|')  # list of author names in that row
    affil = []
    if pd.isnull(row.affiliation):   # if there is no affiliation info, then affil is a list of n nans
        affil = [np.nan]*n
    else:
        affil_list = row.affiliation.split('|') # if there is affiliation info, i build a list of affiliations                       
        
        for idx in row.idx.split('|'): ## list of affiliations indexes for each author , idx is a list of affil indexes for one author                                                           
            
            
            if idx=='-1':
                affil.append('?')
                # OR:
                # affil.append(row.affiliation)
            else:
                try:
                    affil.append('|'.join([affil_list[int(i)] for i in idx.split(',')]))                               
                
                
                except IndexError: 
                    affil.append('?')
   

    
    #return pd.DataFrame({'uid':[row.uid]*n,'author_id':aids,'author_name':names,'affiliation':affil,'seq':range(len(aids))})
    
    # instead, i do:
    return pd.DataFrame({'uid':[row.uid]*n,'author_id':aids,'author_name':names,'affiliation':affil,'seq':range(len(aids))})


result = pd.concat([process(row[1]) for row in  df.iterrows() ] )   # row is a tuple row_index, the entire row info (searchable by:  row[1].author_id  )
#result = pd.concat([process(row[1]) for row in  tqdm_notebook(df.iterrows(), desc='1st loop')   ]     )   # row is a tuple row_index, the entire row info (searchable by:  row[1].author_id  )
print "done"


done


In [ ]:
#  # instead, i do:
#                     string='|'.join([affil_list[int(i)] for i in idx.split(',')])
#                     lista=[affil_list[int(i)] for i in idx.split(',')]
#                     print string
#                     print len(lista) , lista
                    
#                     affil.append(string)
                

In [26]:
len(result)    #487619
result

,affiliation,author_id,author_name,seq,uid
0,"Dokkyo Med Univ, Sch Med, Dept Orthoped Surg, ...",20140647,"Yoshikawa, Katsuhisa",0,WOS:000333129600024
1,"Dokkyo Med Univ, Sch Med, Dept Orthoped Surg, ...",33900476,"Tamai, Kazuya",1,WOS:000333129600024
2,"Dokkyo Med Univ, Sch Med, Dept Orthoped Surg, ...",19778882,"Yano, Yuichiro",2,WOS:000333129600024
3,"Dokkyo Med Univ, Sch Med, Dept Orthoped Surg, ...",8710547,"Kato, Nakayuki",3,WOS:000333129600024
4,"Dokkyo Med Univ, Sch Med, Dept Orthoped Surg, ...",1328775,"Akutsu, Miwa",4,WOS:000333129600024
5,"Dokkyo Med Univ, Sch Med, Dept Orthoped Surg, ...",34499591,"Tomizawa, Kazuo",5,WOS:000333129600024
6,"Dokkyo Med Univ, Sch Med, Dept Orthoped Surg, ...",25003906,"Nohara, Yutaka",6,WOS:000333129600024
0,NaN,9664635,"Bordon, Yvonne",0,WOS:000332939000004
0,"Evelina Childrens Hosp, Dept Gen Paediat, Lond...",45673005,"Lemer, Claire",0,WOS:000333069800001
1,"Imperial Healthcare NHS Trust, Dept Paediat, L...",5470826,"Cheung, Ronny",1,WOS:000333069800001


In [32]:

result=result.loc[result['author_id'] != -1]  # i filter out the authors without desambiguated author id
# i reasign the variable results so i dont eat up all RAM
#df2.head()





print len(result)

486260


In [33]:
print result.affiliation.shape, result.affiliation.dropna().shape

result=result.dropna(subset=['affiliation']).loc[result['affiliation'].dropna().str.contains('USA')]  # i filter out non-USA authors 

print len(result)

(486260,) (473803,)
103241


In [34]:
for row in result.iterrows():
    print row[1].affiliation
   

    raw_input()
    

Univ Texas Austin, Dept Integrat Biol, Austin, TX 78712 USA|Smithsonian Trop Res Inst, Balboa, Panama

Univ Texas Austin, Dept Integrat Biol, Austin, TX 78712 USA|Smithsonian Trop Res Inst, Balboa, Panama

Univ New Mexico, Hlth Sci Ctr, Dept Pathol, Albuquerque, NM 87131 USA

Univ New Mexico, Hlth Sci Ctr, Dept Pathol, Albuquerque, NM 87131 USA

Univ New Mexico, Hlth Sci Ctr, Dept Pathol, Albuquerque, NM 87131 USA

Univ New Mexico, Hlth Sci Ctr, Dept Pathol, Albuquerque, NM 87131 USA

Univ New Mexico, Hlth Sci Ctr, Dept Pathol, Albuquerque, NM 87131 USA

Univ Maryland, Dept Mech Engn, College Pk, MD 20742 USA

NIAID, Lab Malaria & Vector Res, NIH, Bethesda, MD 20892 USA

NIAID, Lab Malaria & Vector Res, NIH, Bethesda, MD 20892 USA

NIAID, Lab Malaria & Vector Res, NIH, Bethesda, MD 20892 USA

Childrens Hosp Montefiore, Bronx, NY USA

Alfred I DuPont Hosp Children, Wilmington, DE USA

Duke Univ, Med Ctr, Durham, NC USA

Univ Penn, Childrens Hosp Philadelphia, Sch Med, Philadelphia, PA 1

KeyboardInterrupt: 

In [ ]:
# #result.sort_values(["author_name", "uid"])
# dictionary={}

# cont =0
# for row in df2.iterrows():  # row is a tuple: row_index, the entire row info (searchable by:  row[1].author_id  )
#     print row[1]
#     print row[1].author_id
#     print row[1].author_name
#     print row[1].affiliation
#     print 
#    # cont +=1
#     #if cont ==10:
#     break

# df2.head()

In [9]:
pub_counts = result.author_id.value_counts()  # it returns a series: index, values  (the values column will have the name of the original column it comes from)
pub_counts.name = 'tot_n_pubs'  # i change the name of the value column of the series
FINAL=result.join(pub_counts,on='author_id') #join: matching INDEX from pub_counts series with author_id column from df2 

FINAL.sort_values("author_id").head(200)
# for 2 dataframes, use merge: df2.merge(df2,on='uid')
#grouped.head(5)


#count_per_author = grouped.Author_ID.value_counts()

#author_data = grouped.groupby('Author_ID').apply(lambda x: x.Author_name.unique())

#author_data[author_data.apply(lambda x: len(x))>1].shape
          #joined = author_data.join(counts_per_author,on='Author_ID')
#new_df.head()
print len(FINAL)

103241


In [10]:
FINAL.sort_values("author_id").head(20)

,affiliation,author_id,author_name,seq,uid,tot_n_pubs
4,"Baylor Coll Med, Houston, TX 77030 USA",96,"Aagaard, Kjersti",4,WOS:000361140900151,3
10,"Baylor Coll Med, Div Maternal Fetal Med, Dept ...",96,"Aagaard, Kjersti M.",10,WOS:000337503800033,3
6,"Baylor Coll Med, Houston, TX 77030 USA",96,"Aagaard, Kjersti",6,WOS:000361140900081,3
0,"Baylor Coll Med, Div Maternal Fetal Med, Houst...",106,"Aagaard, Kjersti",0,WOS:000330322600522,1
3,"AbbVie Inc, N Chicago, IL USA",149,"Anderson, Jaclyn K.",3,WOS:000344384901102,1
0,"Boston Univ, Sch Med, Boston Med Ctr, Boston, ...",187,"Abdullah, Abu S.",0,WOS:000339040600081,1
1,"Duke Univ, Dept Med, Duke Canc Inst, Div Hemat...",478,"Allen, Jessica L.",1,WOS:000357284300020,1
1,"Amer Assisted Living Nurses Assoc, Napa, CA 94...",713,"Allen, Josh",1,WOS:000331595600017,1
1,"US Army Res Lab, Adelphi, MD 20783 USA",733,"Allen, J. L.",1,WOS:000340321300134,1
2,"Univ Minnesota, Sch Med, Duluth, MN 55812 USA",743,"Allen, James",2,WOS:000361758100009,1


In [11]:
# def test(n):
#     for row in FINAL.head(n).iterrows():  # how to do this faster??


#         author_lastname=row[1].author_name.split(",")[0].replace("-"," ").replace("'"," ").capitalize().replace(" ","-").replace(" ","'")
#         try:
#             rest_name=row[1].author_name.split(",")[1].strip()#.replace("-"," ").replace("'"," ").capitalize().replace(" ","-").replace(" ","'")

#             new_name= author_lastname+", "+rest_name

#             row[1].author_name=new_name
#         except: pass


    
def re_capilatizing_lastnames(authorname):
    
    author_lastname=authorname.split(",")[0].replace("-"," ").replace("'"," ").capitalize().replace(" ","-").replace(" ","'")
    try:
         rest_name=authorname.split(",")[1].strip()#.replace("-"," ").replace("'"," ").capitalize().replace(" ","-").replace(" ","'")
   
         new_name= author_lastname+", "+rest_name
             
    except: 
        return authorname
    
    return new_name

%time result = FINAL.author_name.apply(re_capilatizing_lastnames)

## DO NOT USE ITERROWS (MUCH SLOWER THAN .APPLY)

#%time result = test(10000)

#%timeit 2+2


CPU times: user 36 ms, sys: 0 ns, total: 36 ms
Wall time: 30.9 ms


In [12]:
#x=pd.Series([1,2,3])
#x.name='pop'
#x

unique_author_names = FINAL.groupby('author_id').apply(lambda x: x.author_name.unique())
len(unique_author_names)



92386

In [13]:
multiple_author_names=unique_author_names[unique_author_names.apply(lambda x: len(x)>1)]
#len(unique_autaffiliationhor_names[unique_author_names.apply(lambda x: len(x)>1)])


In [1]:
ini_year=str(list_years[0])
final_year=str(list_years[-1])
FINAL.to_csv(path+"final_"+ini_year+"-"+final_year+"_USA.csv", sep='\t')

multiple_author_names.to_csv(path+"lookup_multiple_author_names.csv", sep='\t')



NameError: name 'list_years' is not defined

In [76]:
# # i build a master dict (I DONT ACTUALLY NEED THIS NOW)


# print len(FINAL)
    
# master_dict={}

# for row in FINAL.iterrows():
#     author_id=row[1].author_id
    
#     affil=row[1].affiliation
#     author_name=row[1].author_name
#     paper_id=row[1].uid
#     tot_num_pub=row[1].tot_n_pubs
      
#     author_lastname = author_name.split(',')[0].capitalize()
        
        
    
#     try:
#         master_dict[author_id]
#     except KeyError:        
#         master_dict[author_id]={}
    
    
    
        
#     try:
#         master_dict[author_id]['affiliation'].append(affil)
#     except KeyError:
#         master_dict[author_id]['affiliation']=[]
#         master_dict[author_id]['affiliation'].append(affil)
        
           
#     try:
#         master_dict[author_id]['author_name'].append(author_name)
#     except KeyError:
#         master_dict[author_id]['author_name']=[]
#         master_dict[author_id]['author_name'].append(author_name)
        
             
#     try:
#         master_dict[author_id]['author_lastname'].append(author_lastname)
#     except KeyError:
#         master_dict[author_id]['author_lastname']=[]
#         master_dict[author_id]['author_lastname'].append(author_lastname)
        
        
              
#     try:
#         master_dict[author_id]['paper_id'].append(paper_id)
#     except KeyError:
#         master_dict[author_id]['paper_id']=[]
#         master_dict[author_id]['paper_id'].append(paper_id)
        
        
               
#     master_dict[author_id]['tot_num_pub']=tot_num_pub
        
        
        
        
        
  
# print len(master_dict)

103241
92386


In [83]:
# # i pickle the master dict
# cont_multiple_lastnames_per_id=0.
# for author_id in sorted(master_dict):
#     master_dict[author_id]['author_lastname']= list(set(master_dict[author_id]['author_lastname']))
    
#     if len(master_dict[author_id]['author_lastname']) >1:
#         cont_multiple_lastnames_per_id +=1.
        
        
# # #     if len(master_dict[author_id]['affiliation'])  >1:
# #          print author_id
# #          for llave in master_dict[author_id]:              
# #                  print llave,": ", master_dict[author_id][llave]
#             #x=pd.Series([1,2,3])
# #x.name='pop'


# print "num authors in dict:", len(master_dict), "   fraction of conflicting diff. lastnames with same id:", cont_multiple_lastnames_per_id/len(master_dict)

# filehandler = open( path+"dict_disambig_authors_USA_"+ini_year+"-"+final_year+".pickle","wb")
# %time pickle.dump(master_dict,filehandler)


# # CPU times: user 1.65 s, sys: 28 ms, total: 1.68 s
# # Wall time: 1.68 s


num authors in dict: 92386    fraction of conflicting diff. lastnames with same id: 0.000292252072825
CPU times: user 2.02 s, sys: 36 ms, total: 2.05 s
Wall time: 2.06 s


In [15]:
FINAL.sort_values("author_id").head(10)

,affiliation,author_id,author_name,seq,uid,tot_n_pubs
4,"Baylor Coll Med, Houston, TX 77030 USA",96,"Aagaard, Kjersti",4,WOS:000361140900151,3
10,"Baylor Coll Med, Div Maternal Fetal Med, Dept ...",96,"Aagaard, Kjersti M.",10,WOS:000337503800033,3
6,"Baylor Coll Med, Houston, TX 77030 USA",96,"Aagaard, Kjersti",6,WOS:000361140900081,3
0,"Baylor Coll Med, Div Maternal Fetal Med, Houst...",106,"Aagaard, Kjersti",0,WOS:000330322600522,1
3,"AbbVie Inc, N Chicago, IL USA",149,"Anderson, Jaclyn K.",3,WOS:000344384901102,1
0,"Boston Univ, Sch Med, Boston Med Ctr, Boston, ...",187,"Abdullah, Abu S.",0,WOS:000339040600081,1
1,"Duke Univ, Dept Med, Duke Canc Inst, Div Hemat...",478,"Allen, Jessica L.",1,WOS:000357284300020,1
1,"Amer Assisted Living Nurses Assoc, Napa, CA 94...",713,"Allen, Josh",1,WOS:000331595600017,1
1,"US Army Res Lab, Adelphi, MD 20783 USA",733,"Allen, J. L.",1,WOS:000340321300134,1
2,"Univ Minnesota, Sch Med, Duluth, MN 55812 USA",743,"Allen, James",2,WOS:000361758100009,1


In [20]:
## i get one row per author with all relevant info

def format_affil(affil_string):
    
    return affil_string.replace('Univ,','University,')

def grouping(input_df):
    
    #print "SIZE of input:",len(input_df)
    #author_id=input_df.author_id.iloc[0]   # these two var. should be the same for all rows of the input_df
    tot_n_pubs=input_df.tot_n_pubs.iloc[0]
    
    
    lastname=input_df.author_name.iloc[0].split(",")[0].capitalize()
    
    
    
#     list_affil=[]
#     list_names=[]
#     list_seq=[]
#     list_uid=[]
#     for row in input_df.iterrows():  OJO!!! no need: input_df.affiliation is already a series
       
#         list_affil.append(row[1].affiliation)
#         list_names.append(row[1].author_name)
#         list_seq.append(row[1].seq)
#         list_uid.append(row[1].uid)
        
#     x=["WOS:000331218800012", "WOS:000363776800012"]

#     lista=[x[i].strip("WOS:") for i in range(len(x))]



    #x= input_df.uid.values
    lista=[input_df.uid.values[i].strip("WOS:") for i in range(len(input_df.uid.values))]

    #print input_df.author_id.iloc[0]  
    
  
    #return pd.DataFrame({'author_names':list_names,'affiliations':list_affil,'seqs':list_seq,'tot_n_pub':tot_n_pubs})
    affiliations = [format_affil(a) for a in input_df.affiliation.unique()]
    
    resultado = pd.Series({'author_names':input_df.author_name.unique(),'lastname':lastname,'affiliations':affiliations,\
                         'seqs':input_df.seq.values,'uids':lista,'tot_n_pub':tot_n_pubs}) #input_df.affiliation.unique()
    #if the returned values of some of the columns are list, use SERIES instead of DATAFRAME
    
    
    
    
    return resultado
  #'author_id':author_id,  no need, Univit is the index


#grouped = FINAL.head(10000).groupby('author_groupedid').apply(grouping)
grouped = FINAL.groupby('author_id').apply(grouping)

grouped= grouped.reset_index()  # to reset the indexes (instead of using the authour_id as index, i create a new one)

# most functions do not modify the df in places, i need to do result2 = result.apply etc...



2 ['Baylor Coll Med, Div Maternal Fetal Med, Dept Obstet, Houston, TX 77030 USA|Baylor Coll Med, Div Maternal Fetal Med, Dept Gynecol, Houston, TX 77030 USA|Texas Childrens Hosp, Houston, TX 77030 USA|Baylor Coll Med, Bioinformat Res Lab, Dept Mol & Human Genet, Houston, TX 77030 USA|Baylor Coll Med, Dept Mol & Cell Biol, Houston, TX 77030 USA', 'Baylor Coll Med, Houston, TX 77030 USA'] 


2 ['Baylor Coll Med, Div Maternal Fetal Med, Dept Obstet, Houston, TX 77030 USA|Baylor Coll Med, Div Maternal Fetal Med, Dept Gynecol, Houston, TX 77030 USA|Texas Childrens Hosp, Houston, TX 77030 USA|Baylor Coll Med, Bioinformat Res Lab, Dept Mol & Human Genet, Houston, TX 77030 USA|Baylor Coll Med, Dept Mol & Cell Biol, Houston, TX 77030 USA', 'Baylor Coll Med, Houston, TX 77030 USA'] 


1 ['Baylor Coll Med, Div Maternal Fetal Med, Houston, TX 77030 USA'] 


1 ['AbbVie Inc, N Chicago, IL USA'] 


1 ['Boston University, Sch Med, Boston Med Ctr, Boston, MA 02118 USA|Guangxi Med University, Sch Publ H

KeyboardInterrupt: 

In [115]:
# #remove WOS
# x=["WOS:000331218800012", "WOS:000363776800012"]

# lista=[x[i].strip("WOS:") for i in range(len(x))]
# lista


['000331218800012', '000363776800012']

In [17]:
grouped

,author_id,affiliations,author_names,lastname,seqs,tot_n_pub,uids
0,96,"[Baylor Coll Med, Div Maternal Fetal Med, Dept...","[Aagaard, Kjersti M., Aagaard, Kjersti]",Aagaard,"[10, 6, 4]",3,"[000337503800033, 000361140900081, 00036114090..."
1,106,"[Baylor Coll Med, Div Maternal Fetal Med, Hous...","[Aagaard, Kjersti]",Aagaard,[0],1,[000330322600522]
2,149,"[AbbVie Inc, N Chicago, IL USA]","[Anderson, Jaclyn K.]",Anderson,[3],1,[000344384901102]
3,187,"[Boston University, Sch Med, Boston Med Ctr, B...","[Abdullah, Abu S.]",Abdullah,[0],1,[000339040600081]
4,478,"[Duke University, Dept Med, Duke Canc Inst, Di...","[Allen, Jessica L.]",Allen,[1],1,[000357284300020]
5,713,"[Amer Assisted Living Nurses Assoc, Napa, CA 9...","[Allen, Josh]",Allen,[1],1,[000331595600017]
6,733,"[US Army Res Lab, Adelphi, MD 20783 USA]","[Allen, J. L.]",Allen,[1],1,[000340321300134]
7,743,"[Univ Minnesota, Sch Med, Duluth, MN 55812 USA]","[Allen, James]",Allen,[2],1,[000361758100009]
8,778,"[Univ Virginia, Charlottesville, VA 22904 USA,...","[Allen, Joseph P.]",Allen,"[0, 3]",2,"[000331218800012, 000363776800012]"
9,988,"[RTI Int, Res Triangle Pk, NC 27709 USA]","[Allen, Jane A.]",Allen,[1],1,[000339418300110]


In [20]:
pickle_name=path+"disambig_authors_USA_"+ini_year+"-"+final_year+".pickle"
%time grouped.to_pickle(pickle_name)

CPU times: user 2.32 s, sys: 32 ms, total: 2.35 s
Wall time: 2.36 s


In [101]:
# s="asjdhflkjasd||asdfasd|"

# %timeit  s.replace("||", "|").strip("|")


The slowest run took 6.52 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 622 ns per loop


In [24]:
for row in grouped.iterrows():
    print row[1].author_id
    for item in row[1].affiliations:
        print item,"\n"
    raw_input()
    print 
    print 

96
Baylor Coll Med, Div Maternal Fetal Med, Dept Obstet, Houston, TX 77030 USA|Baylor Coll Med, Div Maternal Fetal Med, Dept Gynecol, Houston, TX 77030 USA|Texas Childrens Hosp, Houston, TX 77030 USA|Baylor Coll Med, Bioinformat Res Lab, Dept Mol & Human Genet, Houston, TX 77030 USA|Baylor Coll Med, Dept Mol & Cell Biol, Houston, TX 77030 USA 

Baylor Coll Med, Houston, TX 77030 USA 




106
Baylor Coll Med, Div Maternal Fetal Med, Houston, TX 77030 USA 




149
AbbVie Inc, N Chicago, IL USA 




187
Boston University, Sch Med, Boston Med Ctr, Boston, MA 02118 USA|Guangxi Med University, Sch Publ Hlth, Nanning, Peoples R China 




478
Duke University, Dept Med, Duke Canc Inst, Div Hematol Malignancies & Cellular Therapy, Durham, NC USA 




713
Amer Assisted Living Nurses Assoc, Napa, CA 94581 USA 




733
US Army Res Lab, Adelphi, MD 20783 USA 




743
Univ Minnesota, Sch Med, Duluth, MN 55812 USA 




778
Univ Virginia, Charlottesville, VA 22904 USA 

Univ Virginia, Charlottesville,

KeyboardInterrupt: 